# Prediction du risque de défaut des clients d'une microfinance à but non lucratif : Cas de Kiva

In [ ]:
# Installation des packages
import pandas as pd
import numpy as np
import os.path
from bs4 import BeautifulSoup 
import lxml
import html5lib
import requests
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from subprocess import check_output
from plotnine import *
import missingno as msno # check missing value
from google.colab import drive
import pycountry_convert as pc
drive.mount('/content/drive')

Dans l'optique de rendre notre travail dynamique, nous avons procéder de deux différentes façon à savoir :

**1 - Télécharger les données de la source à notre ordinateur :** Cette étape présente un inconvénient majeur en terme de temps d'éxécution. Son principal avantage est de rendre automitique l'opération de téléchargement et de dezippage de la base de donnée. En outre elle permet également de rendre dynamique le travail dans la mesure où elle permettra quelque soit l'utilisateur d'avoir la dernière version base donnée de la structure Kiva disponible en ligne. Son procédé est le suivant : scrappé la page web contenant le lien de téléchargement de la base donnée. Par suite étant donné que la base est dans un format zip, nous recourrons aux outils de python pour dézippé la base.

**2 - Utiliser la la base de donnée de donnée de kiva au 20/10/2020**  disponible en dépôt partagé sur drive. Cette méthode est reputée être très rapide et c'est elle qui sera utilisée dans le cadre de cette présentation.

In [ ]:
def importation():
    '''Cette fonction, grâce des techniques de webscrapping et zipfile, permet de télécharger la base directement
        à sur le site de la structure KIVA'''
    
    source = requests.get('https://www.kiva.org/build/data-snapshots').text #Récupération de l'adresse
    soup = BeautifulSoup(source, 'lxml')
    #print(source)
    page = soup.find('div', class_ = 'row page-content') #Scrapping de la page pour rechercher l'élt row page-content dans le div
    #print(json_link.prettify())
    link = []
    
    for i in page.find_all('a'):
        ''' Permet de parcourir l'ensemble des éléments '''
        link.append(i['href'])
    link[1]
    #A ce niveau, nous avons le lien de notre data qui est zippé
    
    csv_link = requests.get(link[1])
    
    if(os.path.exists('dataloa')):  #Test si le dossier existe en local autrement le code a déjà été exécuté
        with open('data.zip','wb') as f:
            f.write(csv_link.content)
        #Extraction
        loan = zipfile.ZipFile('data.zip')
        loan.extractall('data_loan')
        loan.close()
        
        return('data_loan/loans.csv')
    else:                         # Dans ce cas, le dossier existe déjà en local
        nombre_execution += 1     # Va permettre de créer les dossiers en local avec les noms incrémentés
        data = 'data.zip' + str(nombre_execution)
        data_loan = 'data_loan' + str(nombre_execution)
        with open(data,'wb') as f:
            f.write(csv_link.content)
        #Extraction
        loan = zipfile.ZipFile(data)
        loan.extractall(data_loan)
        loan.close()
        
        return(data_loan+'/loans.csv')
# A la fin de cette fonction, notre base de donnée data_new est en dépôt local nous pouvons directement l'utiliser avec pd.read_csv

In [ ]:
loan = pd.read_csv(importation())

In [ ]:
loan = pd.read_csv('/content/drive/My Drive/Stat_App/loans.csv', encoding ='ISO-8859-1',  sep=";")